This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## DeepDream

### Implementing DeepDream in Keras

**Fetching the test image**

In [ ]:
from tensorflow import keras # importing keras from tensorflow
import matplotlib.pyplot as plt # importing matplotlib.pyplot as plt

base_image_path = keras.utils.get_file( # getting the image from the url
    "coast.jpg", origin="https://img-datasets.s3.amazonaws.com/coast.jpg") # url of the image

plt.axis("off") # turning off the axis
plt.imshow(keras.utils.load_img(base_image_path)) # loading the image

**Instantiating a pretrained `InceptionV3` model**

In [ ]:
from tensorflow.keras.applications import inception_v3 # importing inception_v3 from tensorflow.keras.applications
model = inception_v3.InceptionV3(weights="imagenet", include_top=False) # loading the inception_v3 model with imagenet weights and without the top layer of the model (classification layer)

**Configuring the contribution of each layer to the DeepDream loss**

In [ ]:
layer_settings = { # defining the layer settings
    "mixed4": 1.0, # mixed4 layer with 1.0
    "mixed5": 1.5, # mixed5 layer with 1.5
    "mixed6": 2.0, # mixed6 layer with 2.0
    "mixed7": 2.5, # mixed7 layer with 2.5
}
outputs_dict = dict( # defining the outputs_dict
    [
        (layer.name, layer.output) # getting the output of the layer
        for layer in [model.get_layer(name) for name in layer_settings.keys()] # getting the output of the layer for the layer in the model with the name in the layer_settings
    ]
)
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict) # defining the feature_extractor model with the inputs and outputs_dict

**The DeepDream loss**

In [ ]:
def compute_loss(input_image): # defining the compute_loss function with the input_image
    features = feature_extractor(input_image) # extracting the features from the input_image
    loss = tf.zeros(shape=()) # defining the loss as a tensor with shape ()
    for name in features.keys(): # iterating through the names in the features
        coeff = layer_settings[name] # getting the coefficient from the layer_settings
        activation = features[name] # getting the activation from the features
        loss += coeff * tf.reduce_mean(tf.square(activation[:, 2:-2, 2:-2, :])) # adding the coefficient multiplied by the mean of the square of the activation to the loss
    return loss # returning the loss

**The DeepDream gradient ascent process**

In [ ]:
import tensorflow as tf # importing tensorflow as tf

@tf.function # defining the function as a tensorflow function
def gradient_ascent_step(image, learning_rate): # defining the gradient_ascent_step function with the image and learning_rate
    with tf.GradientTape() as tape: # defining the tape
        tape.watch(image) # watching the image
        loss = compute_loss(image) # computing the loss
    grads = tape.gradient(loss, image) # getting the gradients
    grads = tf.math.l2_normalize(grads) # normalizing the gradients
    image += learning_rate * grads # adding the learning_rate multiplied by the gradients to the image
    return loss, image # returning the loss and image


def gradient_ascent_loop(image, iterations, learning_rate, max_loss=None): # defining the gradient_ascent_loop function with the image, iterations, learning_rate, and max_loss
    for i in range(iterations): # iterating through the range of iterations
        loss, image = gradient_ascent_step(image, learning_rate) # getting the loss and image from the gradient_ascent_step function
        if max_loss is not None and loss > max_loss: # if the max_loss is not None and the loss is greater than the max_loss
            break # break the loop
        print(f"... Loss value at step {i}: {loss:.2f}") # print the loss value at step i
    return image # return the image

In [ ]:
# defining the gradient ascent parameters
step = 20. # step is 20
num_octave = 3 # num_octave is 3
octave_scale = 1.4 # octave_scale is 1.4
iterations = 30 # iterations is 30
max_loss = 15. # max_loss is 15.

**Image processing utilities**

In [ ]:
import numpy as np # importing numpy as np

def preprocess_image(image_path): # defining the preprocess_image function with the image_path
    img = keras.utils.load_img(image_path) # loading the image
    img = keras.utils.img_to_array(img) # converting the image to an array
    img = np.expand_dims(img, axis=0) # expanding the dimensions of the image
    img = keras.applications.inception_v3.preprocess_input(img) # preprocessing the image
    return img # returning the image

def deprocess_image(img): # defining the deprocess_image function with the img
    img = img.reshape((img.shape[1], img.shape[2], 3)) # reshaping the image
    img /= 2.0 # dividing the image by 2.0
    img += 0.5 # adding 0.5 to the image
    img *= 255. # multiplying the image by 255.
    img = np.clip(img, 0, 255).astype("uint8") # clipping the image and converting it to an unsigned integer
    return img # returning the image

**Running gradient ascent over multiple successive "octaves"**

In [ ]:
original_img = preprocess_image(base_image_path) # preprocessing the original image
original_shape = original_img.shape[1:3] # getting the original shape of the image

successive_shapes = [original_shape] # defining the successive_shapes with the original_shape
for i in range(1, num_octave): # iterating through the range of 1 to num_octave
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape]) # getting the shape of the image
    successive_shapes.append(shape) # appending the shape to the successive_shapes
successive_shapes = successive_shapes[::-1] # reversing the successive_shapes

shrunk_original_img = tf.image.resize(original_img, successive_shapes[0]) # resizing the original image

img = tf.identity(original_img) # getting the identity of the original image
for i, shape in enumerate(successive_shapes): # iterating through the enumerate of the successive_shapes
    print(f"Processing octave {i} with shape {shape}") # printing the octave and shape 
    img = tf.image.resize(img, shape) # resizing the image
    img = gradient_ascent_loop( # getting the image from the gradient_ascent_loop function
        img, iterations=iterations, learning_rate=step, max_loss=max_loss # with the iterations, learning_rate, and max_loss
    )
    upscaled_shrunk_original_img = tf.image.resize(shrunk_original_img, shape) # resizing the shrunk original image
    same_size_original = tf.image.resize(original_img, shape) # resizing the original image
    lost_detail = same_size_original - upscaled_shrunk_original_img # getting the lost detail
    img += lost_detail # adding the lost detail to the image
    shrunk_original_img = tf.image.resize(original_img, shape) # resizing the original image

keras.utils.save_img("dream.png", deprocess_image(img.numpy())) # saving the image as dream.png

### Wrapping up